In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report
import markovify
import random
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import string
from scipy.stats import entropy
from time import time
from hyperopt import fmin, tpe, hp, Trials
import pickle


In [20]:
#Load passwords that are longer than 8 characters, because most of the time nowadays you can't make a password that is shorter than 8 characters
counter = 0
trainingPass = set()


with open(r"C:\Users\1\Downloads\rockyou.txt", 'r', encoding='latin1') as file:
    for line in file:
        counter +=1
        stripped_line = line.strip()
        if len(trainingPass) > 399999:
            break
        if len(stripped_line) > 7 and counter > 1740247:
            trainingPass.add(stripped_line)
            counter += 1
        




In [3]:
trainingPass = set(trainingPass)
len(trainingPass)

100000

In [4]:
trainingPass = [str(password) for password in trainingPass]


In [2]:
import numpy as np
from scipy.stats import entropy

def compute_custom_features(passwords):
    print(f"Input passwords shape: {passwords.shape}")

    features = []
    for password in passwords.flatten():
        char_counts = np.array([password.count(c) for c in set(password)])
        entropy_val = entropy(char_counts) if len(char_counts) > 1 else 0
        length = len(password)
        lowercase_ratio = sum(1 for c in password if c.islower()) / length
        uppercase_ratio = sum(1 for c in password if c.isupper()) / length
        digit_ratio = sum(1 for c in password if c.isdigit()) / length
        symbol_ratio = sum(1 for c in password if not c.isalnum()) / length
        features.append([entropy_val, lowercase_ratio, uppercase_ratio, digit_ratio, symbol_ratio])
    
    features = np.array(features)
    print(f"Features shape: {features.shape}")
    
    # Convert the dense numeric features to a sparse format
    return csr_matrix(features)  # Return sparse matrix




with open(r"E:\Nasko\PasswordGenerator\rockyou-xato.txt", "r", encoding="utf-8") as f:
       # Join the passwords with spaces to make the model understand transitions between them
    text = " ".join([line.strip() for line in f.readlines() if len(line.strip()) > 7])
    
    model = markovify.Text(text, state_size=2)
    bigrams = list(model.chain.model.keys())
# Function to generate Markov-based passwords and convert to sparse matrix
def generate_markov_password(min_length=8, max_length=16):
    """
    Generate a single password using a Markov model, starting from random n-grams.
    
    Args:
        min_length (int): Minimum length of the generated password.
        max_length (int): Maximum length of the generated password.
    
    Returns:
        str: The generated password.
    """
    # Train a Markov model (only once)
   
    

    # Choose a random bigram from the model's keys to start
    bigram = random.choice(bigrams)
    password = ''.join(bigram)  # Initialize with the bigram
    
    # Generate a random password length between min_length and max_length
    pass_length = random.randint(min_length, max_length)
    
    while len(password) < pass_length:
        state = tuple(password[-2:])  # Last two characters as the new state
        next_char_options = model.chain.model.get(state)
        
        if next_char_options:
            # Select the most frequent next character
            next_char = max(next_char_options, key=next_char_options.get)
            password += next_char[-1]  # Append only the last character
        else:
            # If we hit a dead-end, restart from a new random bigram
            bigram = random.choice(list(model.chain.model.keys()))
            password += ''.join(bigram)[:2]  # Append part of new bigram as fallback

    return password[:pass_length]
# Function to compute entropy and character ratios for custom features
# ColumnTransformer setup







In [6]:
trainingPass = set(trainingPass)
len(trainingPass)
labels = []
for i in range(len(trainingPass)):
    labels.append(1)
while len(trainingPass) < 200000:
    trainingPass.add(generate_markov_password())
for i in range(100000):
    labels.append(0)


In [7]:
print(len(trainingPass))
print(len(labels))

200000
200000


In [8]:
while len(trainingPass) < 300000:
    trainingPass.add(generate_markov_password())

In [9]:
trainingPass = list(trainingPass)
testingData = trainingPass[-100000:]
testingData = list(testingData)

In [10]:
wordlist = set()
with open(r"C:\Users\1\Downloads\hashmob.net.medium.found.txt", encoding= "utf-8") as f:
    for i in f:
        if len(i) > 7:
            wordlist.add(f)
wordlist = list(wordlist)

In [11]:

trainingPass = np.array(trainingPass).reshape(-1,1)
testingData  = np.array(testingData).reshape(-1,1)



In [12]:
print(len(trainingPass))
print(len(labels))
print(len(testingData))

300000
200000
100000


In [13]:
trainingPass = trainingPass[:-100000]

In [14]:
len(trainingPass)

200000

In [ ]:
def compute_dense_features(passwords):
    features = []
    for password in passwords:
        entropy_val = entropy(char_counts) if len(char_counts) > 1 else 0
        length = len(password)
        lowercase_ratio = sum(1 for c in password if c.islower()) / length
        uppercase_ratio = sum(1 for c in password if c.isupper()) / length
        digit_ratio = sum(1 for c in password if c.isdigit()) / length
        symbol_ratio = sum(1 for c in password if not c.isalnum()) / length
        features.append([entropy_val, length, lowercase_ratio, uppercase_ratio, digit_ratio, symbol_ratio])

    return csr_matrix(features)  # Return as sparse matrix


In [ ]:
from hyperopt import hp, tpe, fmin, Trials
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer
import numpy as np
from scipy.stats import entropy
import time
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

# Custom Metric Function
def custom_metric_with_wordlist(estimator, X_test, wordlist, w1=1.0, w2=1.0, w3=1.0):
    start_time = time.time()
    y_pred = estimator.predict(testingData)
    end_time = time.time()

    # Prediction speed (samples per second)
    prediction_speed = len(X_test) / (end_time - start_time)

    # True Positives (TP): Predicted as 1 and in the wordlist
    true_positives = sum((password in wordlist) and (pred == 1) for password, pred in zip(X_test.ravel(), y_pred))
    # True Negatives (TN): Predicted as 0 and not in the wordlist
    true_negatives = sum((password not in wordlist) and (pred == 0) for password, pred in zip(X_test.ravel(), y_pred))
    # False Positives (FP): Predicted as 1 and not in the wordlist
    false_positives = sum((password not in wordlist) and (pred == 1) for password, pred in zip(X_test.ravel(), y_pred))
    # False Negatives (FN): Predicted as 0 and in the wordlist
    false_negatives = sum((password in wordlist) and (pred == 0) for password, pred in zip(X_test.ravel(), y_pred))

    # Ratios
    num_pred_0 = sum(y_pred == 0)
    num_pred_1 = sum(y_pred == 1)
    ratio_pred_1_0 = num_pred_1 / num_pred_0 if num_pred_0 > 0 else 0
    false_positive_rate = false_positives / true_positives if true_positives > 0 else 0
    if ratio_pred_1_0 == 0:
        fistValue = 0
    else:
        fistValue = 1/(w1 * ratio_pred_1_0)
    if false_positive_rate == 0:
        secondValue = 0
    else:
        secondValue = 1/(w2 * false_positive_rate)
    if prediction_speed == 0:
        thirdValue = 0
    else:
        thirdValue = w3 * (1 / prediction_speed)
    
    score =  fistValue + secondValue + thirdValue
    return score  # Negative for minimization

# Compute custom features

# Define the objective function
def objective(params):
    hashing_vectorizer = HashingVectorizer(
        analyzer='char',
        ngram_range=(1, int(params['ngram_range']))
    )
    logistic_regression = LogisticRegression(
        C=params['C'], solver='saga', max_iter=5000,random_state=42
    )

    # Combine vectorizer and custom features
    preprocessor = ColumnTransformer(
        transformers=[
            ('hashing', hashing_vectorizer, 0),
            ('custom_features', FunctionTransformer(compute_dense_features, validate=False), 0)
        ],
        sparse_threshold=1.0
    )

    # Define the pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', logistic_regression)
    ])

    # Split data


    # Train the model
    pipeline.fit(trainingPass, labels)

    # Evaluate using custom metric
    score = custom_metric_with_wordlist(pipeline, testingData, wordlist, w1=1.0, w2=1.0, w3=1.0)
    return score

# Search space for Hyperopt
space = {
    'ngram_range': hp.quniform('ngram_range', 1, 3, 1),  # n-gram range
    'C': hp.loguniform('C', -4, 2)  # Regularization parameter
}

# Example training data


# Run optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=15, trials=trials)

print("Best hyperparameters:", best)


In [16]:
len(labels)

200000

In [27]:
#Load passwords that are longer than 8 characters, because most of the time nowadays you can't make a password that is shorter than 8 characters
counter = 0
trainingPass = set()


with open(r"C:\Users\1\Downloads\rockyou.txt", 'r', encoding='latin1') as file:
    for line in file:
        counter +=1
        stripped_line = line.strip()
        if len(trainingPass) > 399999:
            break
        if len(stripped_line) > 7 and counter > 1740247:
            trainingPass.add(stripped_line)
            counter += 1
        

In [ ]:
with open("generatedPasswordsTestingBigModel.txt", "w",encoding="utf-8") as file:
    for index in testingData:
        # Ensure each entry is a string
        file.write(str(index) + "\n")


In [21]:
len(trainingPass)

400000

In [28]:
trainingPass = set(trainingPass)
len(trainingPass)
labels = []
for i in range(len(trainingPass)):
    labels.append(1)
while len(trainingPass) < 800001:
    trainingPass.add(generate_markov_password())
for i in range(400001):
    labels.append(0)


In [30]:
print(len(trainingPass))
print(len(labels))

800001
800001


In [31]:
with open("PasswordsUsedForTrainingBigModel.txt", "w",encoding="utf-8") as file:
    for index in trainingPass:
        # Ensure each entry is a string
        file.write(str(index) + "\n")


In [32]:
trainingPass = list(trainingPass)
best_params = {
    'C': 3.5888871293577402,
    'ngram_range': 2  # Convert to int if Hyperopt returned it as float
}

# HashingVectorizer with the best n-gram range
hashing_vectorizer = HashingVectorizer(
    analyzer='char',
    ngram_range=(1, int(best_params['ngram_range']))
)

# Logistic Regression with the best regularization parameter
logistic_regression = LogisticRegression(
    C=best_params['C'],
    solver='saga',
    max_iter=5000,
    random_state=42
)

# Combine HashingVectorizer and custom feature transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('hashing', hashing_vectorizer, 0),
        ('custom_features', FunctionTransformer(compute_dense_features, validate=False), 0)
    ],
    sparse_threshold=1.0
)

# Define the final pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', logistic_regression)
])

# Train the model with the best hyperparameters
pipeline.fit(np.array(trainingPass).reshape(-1,1), labels)

# Save the trained model using pickle
with open("better_trained.pkl", "wb") as model_file:
    pickle.dump(pipeline, model_file)

print("Model trained with the best hyperparameters and saved as 'password_classifier_best.pkl'.")

Model trained with the best hyperparameters and saved as 'password_classifier_best.pkl'.


In [ ]:
predictedPasse = pipeline.predict(testingData)

In [ ]:
ones = []
zeros = []
for i in range(len(predictedPasse[i])):
    if predictedPasse[i] == 1:
        ones.append(testModel[i])
    else:
        zerros.appendnd(testModel[i])


In [45]:
with open("Predicted1.txt", "w",encoding="utf-8") as file:
    for index in ones:
        # Ensure each entry is a string
        file.write(str(index) + "\n")
with open("Predicted0.txt", "w",encoding="utf-8") as file:
    for index in zeros:
        # Ensure each entry is a string
        file.write(str(index) + "\n")

In [47]:
len(ones)

50604